# Drones surveillance

A set of drones must be used to supervise an area. 
The area has a rectangular shape and can be discretized as a set of points to supervise.

The following plot is an example of a drone supervising a part of the area:
the dots are the spot we want to supervise, the red ones are the supervised ones, and the blue circle is the area covered by the drone camera.

In [ ]:
import matplotlib.pyplot as plt
import itertools

def draw_grid(canvas,w,h,coord):
    canvas.set_facecolor('#C3CC73')
    canvas.set_ylim([0,h])
    canvas.set_xlim([0,w])
    canvas.set_xticks(range(0,w,1))
    canvas.set_yticks(range(0,h,1))
    canvas.grid()
    grid_dots = list(itertools.product(range(0,w+1), range(0,h+1)))
    gc.scatter([c[0] for c in grid_dots],[c[1] for c in grid_dots], color='#000000', zorder=0)
    gc.scatter([c[0] for c in coord],[c[1] for c in coord], color='#820D10', zorder=2)

def draw_drone(canvas, x, y, radius):
    gc.add_artist(plt.Circle((x,y), radius, zorder=1, color='#516C88'))
    
plt.figure(figsize=(8,8))
gc = plt.gcf().gca()
draw_grid(gc, 10, 10, [(3,3),(3,4),(3,5),(4,3),(4,4),(4,5),(5,3),(5,4),(5,5)])
draw_drone(gc, 4.0, 4.0, 1.7)
plt.show()

The goal is to maximize the supervised area.

## Data

$D:$ set of available drones  
$n,m:$ size in dots of the rectangular area  
$M = \{(i,j) : 0 \le i \le n, 0 \le j \le m, \}: $ set of coordinates  
$r_d:$ surveillance radius of drone $d \in D$  
$\delta(p_1, p_2):$ function computing the euclidean distance between two dots


## variables

$x_{dp}:$ $1$ if drone $d$ is placed in coordinate $p$  
$y_{p}:$ $1$ if coordinate $p$ is supervised

## Objective function

Maximize the supervised area (in terms of supervised dots)
$$z = \max \sum_{p \in M} y_{p}$$

## Constraints

Each drone cannot be placed in two different coordinates
$$ \sum_{p \in M} x_{dp} \le 1, \forall {d \in D}$$

Each dot is supervised only if it within the radius of a drone
$$ y_{p} \le \sum_{d \in D} \sum_{q \in M : \delta(p, q ) \le r_d } x_{dq}, \forall p \in M$$


## Implementation

Creation of the abstrat model

In [ ]:
import pyomo.environ as pyopt

model = pyopt.AbstractModel("Drone surveillance")

The dimensions of the area are set in an arbitrary manner, then the set $M$ is created as the cross product of the sets of the two dimensions of the area:

In [ ]:
m = 15
n = 15
drones = range(6)

model.D = pyopt.Set(initialize=drones)
model.m = pyopt.Set(initialize=range(m))
model.n = pyopt.Set(initialize=range(n))
model.M = model.m * model.n

The only parameter required is the drone radius:

In [ ]:
import random
model.r = pyopt.Param(model.D, initialize={i:random.randrange(2,4) for i in drones})

All variables are binary:

In [ ]:
model.x = pyopt.Var(model.D, model.M, within=pyopt.Binary)
model.y = pyopt.Var(model.M, within=pyopt.Binary)

The objective function is to maximize the number of dots supervised:

In [ ]:
def objective_function(model):
    return pyopt.sum(model.y[i,j] for (i,j) in model.M)

model.z = pyopt.Objective(sense=pyopt.maximize, rule=objective_function)

Constraints `cons_covering` impose that a dot is supervised only if it is within the radius of a drone, while constraints `cons_assignment` avoid to place a drone more than once:

In [ ]:
import math
def delta(x1,y1,x2,y2):
    return math.sqrt( math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2) )

def cons_covering(model, i, j):
    return model.y[i,j] <= pyopt.sum(model.x[d,p,q] 
                                     for d in model.D for (p,q) in model.M 
                                     if delta(i,j,p,q) <= model.r[d] )

model.cons_covering = pyopt.Constraint(model.M, rule=cons_covering)

def cons_assignment(model, d):
    return pyopt.sum(model.x[d,i,j] for (i,j) in model.M) <= 1

model.cons_assignment = pyopt.Constraint(model.D, rule=cons_assignment)

Once the model is complete, we create the instance and solve the problem.

In [ ]:
instance = model.create_instance()
solver = pyopt.SolverFactory('glpk')
results = solver.solve(instance)
print(results)

We can select to display or retrieve only part of the result:

In [ ]:
print(results.Solver.Termination_condition)

In [ ]:
plt.figure(figsize=(8,8))
gc = plt.gcf().gca()
draw_grid(gc, m-1, n-1, [(i,j) for (i,j) in instance.M if instance.y[i,j].value > 0])
for d in instance.D:
    for (i,j) in instance.M:
        if instance.x[d,i,j].value > 0:
            draw_drone(gc, i, j, instance.r[d])
plt.show()